In [0]:
%pip install mlflow==2.10.1 langchain databricks-vectorsearch==0.22 databricks-sdk==0.18.0 mlflow[databricks]
dbutils.library.restartPython()

  Obtaining dependency information for mlflow==2.10.1 from https://files.pythonhosted.org/packages/e5/10/27177dedbe4ea5ffd81bd90a0149d069f6ba431767f490702198c5021cd5/mlflow-2.10.1-py3-none-any.whl.metadata
  Using cached mlflow-2.10.1-py3-none-any.whl.metadata (13 kB)
  Obtaining dependency information for databricks-vectorsearch==0.22 from https://files.pythonhosted.org/packages/81/37/3106dc8dedbab9b7560f65088aa7f2894ffa75b5f1928bc472bf42f3b936/databricks_vectorsearch-0.22-py3-none-any.whl.metadata
  Using cached databricks_vectorsearch-0.22-py3-none-any.whl.metadata (401 bytes)
  Obtaining dependency information for databricks-sdk==0.18.0 from https://files.pythonhosted.org/packages/16/2c/c7920b3806d0c651934a24510f6305958c85df04bacfa65ada3a6cb22e53/databricks_sdk-0.18.0-py3-none-any.whl.metadata
  Using cached databricks_sdk-0.18.0-py3-none-any.whl.metadata (34 kB)
  Obtaining dependency information for mlflow[databricks] from https://files.pythonhosted.org/packages/df/4e/f558c87edb2

In [0]:
%run ./00-init $reset_all_data=false

In [0]:
index_name=f"{catalog}.{db}.vector_search_index"
host = "https://" + spark.conf.get("spark.databricks.workspaceUrl")

In [0]:
# url used to send the request to your model from the serverless endpoint
import os
host = "https://" + spark.conf.get("spark.databricks.workspaceUrl")
os.environ['DATABRICKS_TOKEN'] = dbutils.secrets.get('databricksscope', 'databricks-token') 
#os.environ['DATABRICKS_TOKEN'] = ""

In [0]:
from databricks.vector_search.client import VectorSearchClient
from langchain_community.vectorstores import DatabricksVectorSearch
from langchain_community.embeddings import DatabricksEmbeddings

# Test embedding Langchain model
#NOTE: your question embedding model must match the one used in the chunk in the previous model 
embedding_model = DatabricksEmbeddings(endpoint="databricks-bge-large-en")
print(f"Test embeddings: {embedding_model.embed_query('What is Apache Spark?')[:20]}...")

def get_retriever(persist_dir: str = None):
    os.environ["DATABRICKS_HOST"] = host
    #Get the vector search index
    vsc = VectorSearchClient(workspace_url=host, personal_access_token=os.environ["DATABRICKS_TOKEN"])
    vs_index = vsc.get_index(
        endpoint_name=VECTOR_SEARCH_ENDPOINT_NAME,
        index_name=index_name
    )

    # Create the retriever
    vectorstore = DatabricksVectorSearch(
        vs_index, text_column="content", embedding=embedding_model
    )
    return vectorstore.as_retriever()

# test our retriever
vectorstore = get_retriever()
similar_documents = vectorstore.get_relevant_documents("What is the current Market Sentiment for Zava following its acquisition of Litware and explain the reasons for it?")
print(f"Relevant documents: {similar_documents[0]}")

Test embeddings: [0.0185699462890625, -0.01395416259765625, -0.0576171875, 0.00344085693359375, 0.00856781005859375, -0.021697998046875, -0.024810791015625, -0.00472259521484375, 0.0136260986328125, 0.050323486328125, -0.0275115966796875, -0.01470184326171875, 0.0546875, -0.0538330078125, -0.0102386474609375, -0.0161895751953125, -0.01873779296875, -0.0171356201171875, -0.051177978515625, 0.0179290771484375]...
[NOTICE] Using a Personal Authentication Token (PAT). Recommended for development only. For improved performance, please use Service Principal based authentication. To disable this message, pass disable_notice=True to VectorSearchClient().


Relevant documents: page_content="Quarterly Market Sentiment Report March 2024\nSummary:\nContoso, a retail company headquartered in the US, took a strategic step to improve its market sentiment and customer satisfaction by acquiring LitWare, a European retail company based in the UK. This acquisition has significantly enhanced Contoso's market sentiment score, which has now risen to an impressive 70%. Moreover, the acquisition has resulted in high customer satisfaction and positive reviews, marking a notable turnaround in Contoso's performance.\n© 2023 Contoso Confidential\nPrior to the acquisition, Contoso faced challenges with a market sentiment that was below average compared to industry trends. The lack of focus on Environmental, Social, and Governance (ESG) initiatives led to poor customer satisfaction and lower market sentiment ratings, especially among Millennials, a key market segment for Contoso.\nHowever, with the integration of LitWare, Contoso has successfully addressed th

In [0]:
# Test Databricks Foundation LLM model
from langchain_community.chat_models import ChatDatabricks
chat_model = ChatDatabricks(endpoint="databricks-claude-sonnet-4", max_tokens = 2000)
print(f"Test chat model: {chat_model.predict('What is the current Market Sentiment for Zava following its acquisition of Litware and explain the reasons for it?')}")

/databricks/python/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Test chat model: I don't have access to current market data or recent information about a company called Zava acquiring Litware. To provide you with accurate market sentiment analysis, I would need:

1. **Current market data** - Stock prices, trading volumes, analyst ratings
2. **Recent news and reports** - Press releases, financial filings, analyst coverage
3. **Company details** - Which specific companies these are, as there may be multiple companies with similar names

To get the current market sentiment, I'd recommend:

- **Financial news sources**: Bloomberg, Reuters, Financial Times
- **Stock analysis platforms**: Yahoo Finance, Google Finance, MarketWatch
- **Analyst reports**: From investment banks and research firms
- **Company investor relations**: Official press releases and earnings calls

If you can provide more context about which specific companies you're referring to, or if you have access to recent news about this acquisition, I'd be happy to help analyze what factors 

In [0]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatDatabricks

TEMPLATE = """You are an assistant for Databricks users. You are answering Zava's market sentiment churn and Market sentiments. If the question is not related to one of these topics, kindly decline to answer. If you don't know the answer, just say that you don't know, don't try to make up an answer. Keep the answer as concise as possible.
Use the following pieces of context to answer the question at the end:
{context}
Question: {question}
Answer:
"""
prompt = PromptTemplate(template=TEMPLATE, input_variables=["context", "question"])

chain = RetrievalQA.from_chain_type(
    llm=chat_model,
    chain_type="stuff",
    retriever=get_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

[NOTICE] Using a Personal Authentication Token (PAT). Recommended for development only. For improved performance, please use Service Principal based authentication. To disable this message, pass disable_notice=True to VectorSearchClient().


In [0]:
# langchain.debug = True #uncomment to see the chain details and the full prompt being sent
question = {"query": "What is the current Market Sentiment for Zava following its acquisition of Litware and explain the reasons for it?"}
answer = chain.run(question)
print(answer)


/databricks/python/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


I don't see any information about Zava's market sentiment in the provided context. The documents discuss Contoso's market sentiment following its acquisition of LitWare, but there's no mention of a company called Zava or its acquisition activities. 

If you're asking about Contoso instead, I'd be happy to provide that information based on the available data.


In [0]:
from mlflow.models import infer_signature
import mlflow
import langchain

mlflow.set_registry_uri("databricks-uc")
model_name = f"{catalog}.{db}.rag_chatbot_model"

with mlflow.start_run(run_name="chatbot_rag") as run:
    signature = infer_signature(question, answer)
    model_info = mlflow.langchain.log_model(
        chain,
        loader_fn=get_retriever,  # Load the retriever with DATABRICKS_TOKEN env as secret (for authentication).
        artifact_path="chain",
        registered_model_name=model_name,
        pip_requirements=[
            "mlflow==" + mlflow.__version__,
            "langchain==" + langchain.__version__,
            "databricks-vectorsearch",
        ],
        input_example=question,
        signature=signature
    )

/local_disk0/.ephemeral_nfs/envs/pythonEnv-200d17e3-bae9-4198-b5c2-07f31a82489d/lib/python3.11/site-packages/mlflow/langchain/utils/__init__.py:670: UserWarning: Using custom serializer to pickle pydantic.fields.ModelField classes, this might miss some fields and validators. To avoid this, please upgrade pydantic to v2 using `pip install pydantic -U` with langchain 0.0.267 and above.
  warnings.warn(
2025/07/31 17:46:38 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.10.1, required: mlflow==2.14.3)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.
/databricks/python/lib/python3.11/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


Uploading artifacts:   0%|          | 0/11 [00:00<?, ?it/s]

Successfully registered model 'litware_unity_catalog.rag.rag_chatbot_model'.


Uploading artifacts:   0%|          | 0/11 [00:00<?, ?it/s]

Created version '1' of model 'litware_unity_catalog.rag.rag_chatbot_model'.


In [0]:
model = mlflow.langchain.load_model(model_info.model_uri)
model.invoke(question)

/local_disk0/.ephemeral_nfs/envs/pythonEnv-200d17e3-bae9-4198-b5c2-07f31a82489d/lib/python3.11/site-packages/mlflow/langchain/utils/__init__.py:670: UserWarning: Using custom serializer to pickle pydantic.fields.ModelField classes, this might miss some fields and validators. To avoid this, please upgrade pydantic to v2 using `pip install pydantic -U` with langchain 0.0.267 and above.
  warnings.warn(


[NOTICE] Using a Personal Authentication Token (PAT). Recommended for development only. For improved performance, please use Service Principal based authentication. To disable this message, pass disable_notice=True to VectorSearchClient().


/databricks/python/lib/python3.11/site-packages/langchain_community/llms/__init__.py:173: LangChainDeprecationWarning: `` was deprecated in LangChain 0.0.22 and will be removed in 0.3. An updated version of the  exists in the langchain-community package and should be used instead. To use it run `pip install -U langchain-community` and import as `from langchain_community.chat_models import ChatDatabricks`.
  warn_deprecated(


{'query': 'What is the current Market Sentiment for Zava following its acquisition of Litware and explain the reasons for it?',
 'result': "I don't see any information about Zava's market sentiment in the provided context. The documents discuss Contoso's market sentiment following its acquisition of LitWare, but there's no mention of a company called Zava or its acquisition activities. \n\nIf you're asking about Contoso instead, I'd be happy to provide that information based on the available data."}

### Deploying our Chat Model as a Serverless Model Endpoint


In [0]:
import mlflow
from mlflow.tracking import MlflowClient
from databricks.sdk.service.serving import EndpointCoreConfigInput, ServedModelInput, ServedModelInputWorkloadSize

In [0]:
mlflow.set_registry_uri('databricks-uc')
model_name = f"{catalog}.{db}.rag_chatbot_model"
serving_endpoint_name = "rag-chatbot-model-endpoint"
latest_model_version = 1

In [0]:
# Create or update serving endpoint
from databricks.sdk import WorkspaceClient
from databricks.sdk.service.serving import EndpointCoreConfigInput, ServedModelInput, ServedModelInputWorkloadSize

endpoint_config = EndpointCoreConfigInput(
    name=serving_endpoint_name,
    served_models=[
        ServedModelInput(
            model_name=model_name,
            model_version=latest_model_version,
            workload_size=ServedModelInputWorkloadSize.LARGE,
            scale_to_zero_enabled=True,
            environment_vars={
                "DATABRICKS_TOKEN": "{{secrets/databricksscope/databricks-token}}" ,
            }
        )
    ]
)

In [0]:
from databricks.sdk import WorkspaceClient
w = WorkspaceClient()

existing_endpoint = next(
    (e for e in w.serving_endpoints.list() if e.name == serving_endpoint_name), None
)
serving_endpoint_url = f"{host}/ml/endpoints/{serving_endpoint_name}"
if existing_endpoint == None:
    print(f"Creating the endpoint {serving_endpoint_url}, this will take a few minutes to package and deploy the endpoint...")
    w.serving_endpoints.create_and_wait(name=serving_endpoint_name, config=endpoint_config)
else:
    print(f"Updating the endpoint {serving_endpoint_url} to version {latest_model_version}, this will take a few minutes to package and deploy the endpoint...")
    w.serving_endpoints.update_config_and_wait(served_models=endpoint_config.served_models, name=serving_endpoint_name)

Creating the endpoint https://adb-2273246882692142.2.azuredatabricks.net/ml/endpoints/rag-chatbot-model-endpoint, this will take a few minutes to package and deploy the endpoint...
